# Setup

In [1]:
import sys
sys.path.append("../")

import random
import copy
import os
from collections import namedtuple

import torch
import torch.nn as nn

from torchvision.models import *
from torchvision.transforms import *
from torch.utils.data import DataLoader
from torch.optim import *
from torch.optim.lr_scheduler import *

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from Dataset.dataset import *
from Dataset.data_generation import ArrhythmiaLabels
from Utils.classification import *
from Utils.model_loading import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

d:\Anaconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda:0


# Training Different Models

- IMPORTANT: Replace the output layer correctly before training or face suffering

In [2]:
image_size = 224
dataloader = build_dataloader(
    train_path = "Data/mitbih_mif_train_small.h5",
    test_path  = "Data/mitbih_mif_test.h5",
    batch_size = 32,
    transform  = Resize((image_size, image_size), antialias=None)
)

# visualize_ecg_data(dataloader["train"])

In [3]:
model_name = "vgg16_bn"
model = load_model(model_name)

# Replace classification layer output
num_classes = ArrhythmiaLabels.size
model.classifier[6] = nn.Linear(4096, num_classes)

# Finetuning
num_finetune_epochs = 50
model.to(device)
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, num_finetune_epochs)
criterion = nn.CrossEntropyLoss()

best_model_checkpoint = dict()
best_accuracy = 0
safety = True

if not os.path.exists("Pretrained"):
    os.makedirs("Pretrained")

print("Finetuning")
for epoch in range(num_finetune_epochs):
    train(model, dataloader["train"], criterion, optimizer, scheduler)
    accuracy = evaluate(model, dataloader["test"])

    if accuracy > best_accuracy:
        best_model_checkpoint["state_dict"] = copy.deepcopy(model.state_dict())
        best_accuracy = accuracy
    
    if safety and (epoch + 1) % 10 == 0:
        checkpoint = {
            "model_state_dict":     model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "epoch":                epoch,
        }
        if not os.path.exists("Pretrained/Safety"):
            os.makedirs("Pretrained/Safety")
        torch.save(checkpoint, f"Pretrained/Safety/check_{epoch+1}.pth")
    
    print(f"Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%")

# Save best model    
save_path = f"Pretrained/{model_name}_ecg_ep{num_finetune_epochs}_i{image_size}.pth"
torch.save(best_model_checkpoint["state_dict"], save_path)

# Test saved model
model = load_model_from_pretrained(model_name, save_path, num_classes=num_classes)
model.to(device)
acc = evaluate(model, dataloader["test"])
print(f"Accuracy of Loaded Model: {acc:.2f}")

Finetuning


Epoch 1 Accuracy 97.36% / Best Accuracy: 97.36%


Epoch 2 Accuracy 95.74% / Best Accuracy: 97.36%


Epoch 3 Accuracy 97.03% / Best Accuracy: 97.36%


Epoch 4 Accuracy 97.83% / Best Accuracy: 97.83%


...

Epoch 47 Accuracy 98.53% / Best Accuracy: 98.61%


Epoch 48 Accuracy 98.49% / Best Accuracy: 98.61%


Epoch 49 Accuracy 98.56% / Best Accuracy: 98.61%


Epoch 50 Accuracy 98.63% / Best Accuracy: 98.63%


Accuracy of Loaded Model: 98.63


# Resolution Scaling

In [4]:
def train_with_loss_record(
    model:      nn.Module,
    dataloader: DataLoader,
    criterion:  nn.Module,
    optimizer:  Optimizer,
    scheduler:  LambdaLR
) -> list:
    model.train()
    running_loss = []

    for inputs, labels in tqdm(dataloader, desc="Train", leave=False):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss.append(loss.item())

        loss.backward()

        optimizer.step()
        scheduler.step()

    return running_loss

In [ ]:
model_name  = "mobilenet_v3_small"
num_classes = ArrhythmiaLabels.size
num_finetune_epochs = 15
resolutions = [80, 104, 128, 152, 176, 200, 224]
running_accuracy = {r: [] for r in resolutions}
running_loss = {l: [] for l in resolutions}

for resolution in resolutions:
    # Setup training
    model = load_model(model_name)
    model.classifier[3] = nn.Linear(1024, num_classes)
    model.to(device)

    dataloader = build_dataloader(
        train_path = "Data/mitbih_mif_train_small.h5",
        test_path  = "Data/mitbih_mif_test.h5",
        batch_size = 128,
        transform  = Resize((resolution, resolution))
    )
    
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, num_finetune_epochs)
    criterion = nn.CrossEntropyLoss()

    best_model_checkpoint = dict()
    best_accuracy = 0

    # Finetune
    print(f"Finetuning at resolution {resolution}")
    for epoch in range(num_finetune_epochs):
        losses = train_with_loss_record(model, dataloader["train"], criterion, optimizer, scheduler)
        accuracy = evaluate(model, dataloader["test"])
        
        running_accuracy[resolution].append(accuracy)
        running_loss[resolution].append(losses)

        if accuracy > best_accuracy:
            best_model_checkpoint["state_dict"] = copy.deepcopy(model.state_dict())
            best_accuracy = accuracy
        
        print(f"Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%")

    # Save best model
    if not os.path.exists("Pretrained/MobileNetV3-Small"):
        os.makedirs("Pretrained/MobileNetV3-Small")
    
    save_path = f"Pretrained/MobileNetV3-Small/{model_name}_ecg_ep{num_finetune_epochs}_i{resolution}.pth"
    torch.save(best_model_checkpoint["state_dict"], save_path)

    # Write the running accuracy and loss to another file for safety
    with open("running_acc.txt", "a") as file:
        file.write(" ".join(map(str, running_accuracy[resolution])))
        file.write("\n")
    
    with open("running_loss.txt", "a") as file:
        total_losses = []
        for epoch_losses in running_loss[resolution]:
            total_losses += epoch_losses

        file.write(" ".join(map(str, total_losses)))
        file.write("\n")

# OFA

In [ ]:
ofa_specialized_get = torch.hub.load("mit-han-lab/once-for-all", "ofa_specialized_get")

In [12]:
model, image_size = ofa_specialized_get("pixel1_lat@20ms_top1@71.4_finetune@25", pretrained=True)

dataloader = build_dataloader(
    train_path = "Data/mitbih_mif_train_small.h5",
    test_path  = "Data/mitbih_mif_test.h5",
    batch_size = 128,
    transform  = Resize((152, 152))
)

model.classifier.linear = nn.Linear(1280, 5)

In [ ]:
model_name  = "ofa_pixel1_20"
num_classes = ArrhythmiaLabels.size

# Finetuning
num_finetune_epochs = 50
model.to(device)
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, num_finetune_epochs)
criterion = nn.CrossEntropyLoss()

best_model_checkpoint = dict()
best_accuracy = 0

print("Finetuning")
for epoch in range(num_finetune_epochs):
    train(model, dataloader["train"], criterion, optimizer, scheduler)
    accuracy = evaluate(model, dataloader["test"])

    if accuracy > best_accuracy:
        best_model_checkpoint["state_dict"] = copy.deepcopy(model.state_dict())
        best_accuracy = accuracy
    
    print(f"Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%")

# Save best model
if not os.path.exists("Pretrained"):
    os.makedirs("Pretrained")
    
save_path = f"Pretrained/{model_name}_ecg_ep{num_finetune_epochs}_i{image_size}.pth"
torch.save(best_model_checkpoint["state_dict"], save_path)

In [ ]:
# Test saved model
model, _ = ofa_specialized_get("flops@595M_top1@80.0_finetune@75", pretrained=True)
model.classifier.linear = nn.Linear(1536, 5)
model.load_state_dict(torch.load("Pretrained\ofa_595M_ecg_ep50_i152.pth"))
model.to(device)
acc = evaluate(model, dataloader["test"])
print(f"Accuracy of Loaded Model: {acc:.2f}")

# Raw Dataset Training

## Base Model

In [7]:
image_sizes = [128]

for image_size in image_sizes:
    dataloader = build_dataloader(
        train_path    = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{image_size}/image_full_unfiltered_i{image_size}_train.h5",
        test_path     = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{image_size}/image_full_unfiltered_i{image_size}_test.h5",
        transform     = None,
        batch_size    = 128,
        preload_train = True
    )

    num_classes = ArrhythmiaLabels.size
    model_name  = "resnet50"
    model       = load_model(model_name)
    model.fc    = nn.Linear(2048, num_classes)

    epochs    = 50
    save_name = f"{model_name}_ep{epochs}_i{image_size}_full"
    save_dir  = f"../Pretrained/ECG-Raw/Base/{save_name}"

    warm_up_dataloader(dataloader["train"])
    warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth"
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

[2024-07-16 11:07:41,429] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_full_unfiltered_i128_train.h5
[2024-07-16 11:07:41,431] [DTSET] [INFO] 	Available Memory: 21.99 GiB
[2024-07-16 11:07:41,431] [DTSET] [INFO] 	Dataset Memory:   16.04 GiB
[2024-07-16 11:07:41,431] [DTSET] [INFO] 	Preloading images and labels
[2024-07-16 11:09:40,627] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_full_unfiltered_i128_test.h5
[2024-07-16 11:09:40,628] [DTSET] [INFO] 	Available Memory: 6.98 GiB
[2024-07-16 11:09:40,629] [DTSET] [INFO] 	Dataset Memory:   4.01 GiB
[2024-07-16 11:10:33,199] [CLASS] [INFO] Begin finetuning...
[2024-07-16 11:11:52,955] [CLASS] [INFO] 	Epoch 1 Accuracy 97.98% / Best Accuracy: 97.98%. Time: 79.75434231758118s
[2024-07-16 11:13:06,974] [CLASS] [INFO] 	Epoch 2 Accuracy 98.46% / Best Accuracy: 98.46%. Time: 74.01820039749146s
[2024-07-16 11:14:21,267] [CLASS] [INFO] 	Epoch 3 Accuracy 98.45%

## Dataset Shrinking

In [2]:
for ratio in [0.7, 0.6, 0.5, 0.4, 0.3]:
    for method in ["balance", "random"]:
        image_size = 128
        dataloader = build_dataloader(
            train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{image_size}/image_unfiltered_i{image_size}_train_r{ratio}_{method}.h5",
            test_path  = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{image_size}/image_unfiltered_i{image_size}_test.h5",
            transform  = None,
            batch_size = 128
        )

        num_classes = ArrhythmiaLabels.size
        model_name  = "resnet18"
        model       = load_model(model_name)
        model.fc    = nn.Linear(512, num_classes)

        epochs    = 50
        save_name = f"{model_name}_ep{epochs}_i{image_size}_r{ratio}_{method}"
        save_dir  = f"../Pretrained/ECG-Raw/Dataset-Shrinking/{save_name}"

        warm_up_dataloader(dataloader["train"])
        warm_up_dataloader(dataloader["test"])

        training_stats = finetune(
            model      = model,
            epochs     = epochs,
            dataloader = dataloader,
            save_path  = f"{save_dir}/{save_name}.pth"
        )

        training_stats.serialize(f"{save_dir}/stats.txt")

[2024-07-03 23:26:35,985] [UTL] [INFO] Begin finetuning...
[2024-07-03 23:26:57,270] [UTL] [INFO] 	Epoch 1 Accuracy 98.51% / Best Accuracy: 98.51%
[2024-07-03 23:27:17,599] [UTL] [INFO] 	Epoch 2 Accuracy 98.42% / Best Accuracy: 98.51%
[2024-07-03 23:27:37,931] [UTL] [INFO] 	Epoch 3 Accuracy 98.42% / Best Accuracy: 98.51%
[2024-07-03 23:27:58,440] [UTL] [INFO] 	Epoch 4 Accuracy 98.62% / Best Accuracy: 98.62%
[2024-07-03 23:28:18,840] [UTL] [INFO] 	Epoch 5 Accuracy 98.61% / Best Accuracy: 98.62%
[2024-07-03 23:28:39,317] [UTL] [INFO] 	Epoch 6 Accuracy 98.71% / Best Accuracy: 98.71%
[2024-07-03 23:28:59,741] [UTL] [INFO] 	Epoch 7 Accuracy 98.81% / Best Accuracy: 98.81%
[2024-07-03 23:29:20,254] [UTL] [INFO] 	Epoch 8 Accuracy 98.89% / Best Accuracy: 98.89%
[2024-07-03 23:29:40,620] [UTL] [INFO] 	Epoch 9 Accuracy 98.79% / Best Accuracy: 98.89%
[2024-07-03 23:30:01,093] [UTL] [INFO] 	Epoch 10 Accuracy 98.87% / Best Accuracy: 98.89%
[2024-07-03 23:30:21,631] [UTL] [INFO] 	Epoch 11 Accuracy 98

## Window Shrinking

### Resize and Cropping

In [3]:
for method in ["resize", "crop"]:
    dataloader = build_dataloader(
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_{method}_i128_train.h5",
        test_path  = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_{method}_i128_test.h5",
        transform  = None,
        batch_size = 128
    )

    num_classes = ArrhythmiaLabels.size
    model_name  = "resnet18"
    model       = load_model(model_name)
    model.fc    = nn.Linear(512, num_classes)

    epochs    = 50
    save_name = f"{model_name}_ep{epochs}_i128_{method}"
    save_dir  = f"../Pretrained/ECG-Raw/Base/{save_name}"

    warm_up_dataloader(dataloader["train"])
    warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth"
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

[2024-07-08 19:00:56,614] [UTL] [INFO] Begin finetuning...
[2024-07-08 19:01:29,706] [UTL] [INFO] 	Epoch 1 Accuracy 98.73% / Best Accuracy: 98.73%. Time: 33.091649293899536s
[2024-07-08 19:01:57,099] [UTL] [INFO] 	Epoch 2 Accuracy 98.92% / Best Accuracy: 98.92%. Time: 27.39310336112976s
[2024-07-08 19:02:24,701] [UTL] [INFO] 	Epoch 3 Accuracy 98.95% / Best Accuracy: 98.95%. Time: 27.601943969726562s
[2024-07-08 19:02:52,311] [UTL] [INFO] 	Epoch 4 Accuracy 98.88% / Best Accuracy: 98.95%. Time: 27.609546661376953s
[2024-07-08 19:03:19,829] [UTL] [INFO] 	Epoch 5 Accuracy 99.06% / Best Accuracy: 99.06%. Time: 27.51664113998413s
[2024-07-08 19:03:47,361] [UTL] [INFO] 	Epoch 6 Accuracy 99.14% / Best Accuracy: 99.14%. Time: 27.531224966049194s
[2024-07-08 19:04:14,985] [UTL] [INFO] 	Epoch 7 Accuracy 99.08% / Best Accuracy: 99.14%. Time: 27.62355327606201s
[2024-07-08 19:04:42,608] [UTL] [INFO] 	Epoch 8 Accuracy 99.13% / Best Accuracy: 99.14%. Time: 27.6229829788208s
[2024-07-08 19:05:10,315] 

In [3]:
dataloader = build_dataloader(
    train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_train.h5",
    test_path  = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5",
    transform  = None,
    batch_size = 128
)

num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
model = load_model_from_pretrained(model_name, "../Pretrained/ECG-Raw/Base/resnet18_ep50_i128_resize/resnet18_ep50_i128_resize.pth", num_classes)

epochs    = 30
save_name = f"{model_name}_ep{epochs}_i128_resize_back"
save_dir  = f"../Pretrained/ECG-Raw/Base/{save_name}"

warm_up_dataloader(dataloader["train"])
warm_up_dataloader(dataloader["test"])

training_stats = finetune(
    model      = model,
    epochs     = epochs,
    dataloader = dataloader,
    save_path  = f"{save_dir}/{save_name}.pth"
)

training_stats.serialize(f"{save_dir}/stats.txt")

[2024-07-08 21:46:49,871] [UTL] [INFO] Begin finetuning...
[2024-07-08 21:47:18,571] [UTL] [INFO] 	Epoch 1 Accuracy 98.47% / Best Accuracy: 98.47%. Time: 28.69891667366028s
[2024-07-08 21:47:46,198] [UTL] [INFO] 	Epoch 2 Accuracy 98.63% / Best Accuracy: 98.63%. Time: 27.626131772994995s
[2024-07-08 21:48:13,760] [UTL] [INFO] 	Epoch 3 Accuracy 98.85% / Best Accuracy: 98.85%. Time: 27.562315464019775s
[2024-07-08 21:48:41,331] [UTL] [INFO] 	Epoch 4 Accuracy 98.75% / Best Accuracy: 98.85%. Time: 27.57027840614319s
[2024-07-08 21:49:08,955] [UTL] [INFO] 	Epoch 5 Accuracy 98.70% / Best Accuracy: 98.85%. Time: 27.62327265739441s
[2024-07-08 21:49:36,595] [UTL] [INFO] 	Epoch 6 Accuracy 98.79% / Best Accuracy: 98.85%. Time: 27.640103101730347s
[2024-07-08 21:50:04,366] [UTL] [INFO] 	Epoch 7 Accuracy 98.78% / Best Accuracy: 98.85%. Time: 27.770612716674805s
[2024-07-08 21:50:32,218] [UTL] [INFO] 	Epoch 8 Accuracy 98.91% / Best Accuracy: 98.91%. Time: 27.85117816925049s
[2024-07-08 21:51:00,122]

In [3]:
num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
model       = load_model(model_name)
model.fc    = nn.Linear(512, num_classes)

# To resume training
# model = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/resnet18_ep10_i256_r0.3/resnet18_ep10_i256_r0.3.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [
    TrainConfig(256, 0.3, 10),
    TrainConfig(224, 0.3, 10),
    TrainConfig(192, 0.4, 10),
    TrainConfig(160, 0.5, 10),
    TrainConfig(128, 1.0, 30),
]
method = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")

    if config.window_size == 128:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_resize_i{config.window_size}_train.h5"
    else:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_resize_i{config.window_size}_train_r{config.ratio}.h5"

    dataloader = build_dataloader(
        train_path = train_path,
        test_path  = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_resize_i128_test.h5",
        transform  = None,
        batch_size = 128
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Resize/{save_name}"

    warm_up_dataloader(dataloader["train"])
    if config.window_size == 128:
        warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = (config.window_size == 128) # Only evaluate on 128
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

===== Window Size 256 =====


[2024-07-10 10:52:07,452] [UTL] [INFO] Begin finetuning...
[2024-07-10 10:52:36,681] [UTL] [INFO] 	Epoch 1 Finished. Time: 29.22675085067749s
[2024-07-10 10:53:02,037] [UTL] [INFO] 	Epoch 2 Finished. Time: 25.35603642463684s
[2024-07-10 10:53:27,475] [UTL] [INFO] 	Epoch 3 Finished. Time: 25.43710970878601s
[2024-07-10 10:53:52,975] [UTL] [INFO] 	Epoch 4 Finished. Time: 25.500168085098267s
[2024-07-10 10:54:18,513] [UTL] [INFO] 	Epoch 5 Finished. Time: 25.53720188140869s
[2024-07-10 10:54:44,056] [UTL] [INFO] 	Epoch 6 Finished. Time: 25.543205976486206s
[2024-07-10 10:55:09,685] [UTL] [INFO] 	Epoch 7 Finished. Time: 25.629284858703613s
[2024-07-10 10:55:35,342] [UTL] [INFO] 	Epoch 8 Finished. Time: 25.65731143951416s
[2024-07-10 10:56:01,011] [UTL] [INFO] 	Epoch 9 Finished. Time: 25.668319940567017s
[2024-07-10 10:56:26,671] [UTL] [INFO] 	Epoch 10 Finished. Time: 25.660313606262207s
[2024-07-10 10:56:26,671] [UTL] [INFO] Finetuning completed
[2024-07-10 10:56:26,812] [UTL] [INFO] Model 

===== Window Size 224 =====


[2024-07-10 11:04:20,434] [UTL] [INFO] Begin finetuning...
[2024-07-10 11:04:40,298] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.863122940063477s
[2024-07-10 11:04:59,882] [UTL] [INFO] 	Epoch 2 Finished. Time: 19.582791090011597s
[2024-07-10 11:05:19,504] [UTL] [INFO] 	Epoch 3 Finished. Time: 19.621828317642212s
[2024-07-10 11:05:39,189] [UTL] [INFO] 	Epoch 4 Finished. Time: 19.685885429382324s
[2024-07-10 11:05:58,907] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.7169132232666s
[2024-07-10 11:06:18,637] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.72892475128174s
[2024-07-10 11:06:38,374] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.736931562423706s
[2024-07-10 11:06:58,160] [UTL] [INFO] 	Epoch 8 Finished. Time: 19.785975694656372s
[2024-07-10 11:07:17,951] [UTL] [INFO] 	Epoch 9 Finished. Time: 19.78998112678528s
[2024-07-10 11:07:37,752] [UTL] [INFO] 	Epoch 10 Finished. Time: 19.800989627838135s
[2024-07-10 11:07:37,752] [UTL] [INFO] Finetuning completed
[2024-07-10 11:07:37,860] [UTL] [INFO] Model

===== Window Size 192 =====


[2024-07-10 11:17:07,693] [UTL] [INFO] Begin finetuning...
[2024-07-10 11:17:27,509] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.81642436981201s
[2024-07-10 11:17:46,774] [UTL] [INFO] 	Epoch 2 Finished. Time: 19.263499975204468s
[2024-07-10 11:18:06,111] [UTL] [INFO] 	Epoch 3 Finished. Time: 19.336567163467407s
[2024-07-10 11:18:25,454] [UTL] [INFO] 	Epoch 4 Finished. Time: 19.342573404312134s
[2024-07-10 11:18:44,847] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.392619371414185s
[2024-07-10 11:19:04,239] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.392618894577026s
[2024-07-10 11:19:23,698] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.457677364349365s
[2024-07-10 11:19:43,180] [UTL] [INFO] 	Epoch 8 Finished. Time: 19.481700897216797s
[2024-07-10 11:20:02,644] [UTL] [INFO] 	Epoch 9 Finished. Time: 19.46468472480774s
[2024-07-10 11:20:22,163] [UTL] [INFO] 	Epoch 10 Finished. Time: 19.51873207092285s
[2024-07-10 11:20:22,163] [UTL] [INFO] Finetuning completed
[2024-07-10 11:20:22,253] [UTL] [INFO] Mode

===== Window Size 160 =====


[2024-07-10 11:31:49,212] [UTL] [INFO] Begin finetuning...
[2024-07-10 11:32:07,236] [UTL] [INFO] 	Epoch 1 Finished. Time: 18.022772550582886s
[2024-07-10 11:32:24,888] [UTL] [INFO] 	Epoch 2 Finished. Time: 17.652036905288696s
[2024-07-10 11:32:42,608] [UTL] [INFO] 	Epoch 3 Finished. Time: 17.719098567962646s
[2024-07-10 11:33:00,382] [UTL] [INFO] 	Epoch 4 Finished. Time: 17.773147344589233s
[2024-07-10 11:33:18,177] [UTL] [INFO] 	Epoch 5 Finished. Time: 17.794167518615723s
[2024-07-10 11:33:35,967] [UTL] [INFO] 	Epoch 6 Finished. Time: 17.790161848068237s
[2024-07-10 11:33:53,814] [UTL] [INFO] 	Epoch 7 Finished. Time: 17.845213890075684s
[2024-07-10 11:34:11,685] [UTL] [INFO] 	Epoch 8 Finished. Time: 17.871236085891724s
[2024-07-10 11:34:29,574] [UTL] [INFO] 	Epoch 9 Finished. Time: 17.889252424240112s
[2024-07-10 11:34:47,443] [UTL] [INFO] 	Epoch 10 Finished. Time: 17.868233680725098s
[2024-07-10 11:34:47,443] [UTL] [INFO] Finetuning completed
[2024-07-10 11:34:47,510] [UTL] [INFO] M

===== Window Size 128 =====


[2024-07-10 11:55:36,421] [UTL] [INFO] Begin finetuning...
[2024-07-10 11:56:04,686] [UTL] [INFO] 	Epoch 1 Accuracy 99.04% / Best Accuracy: 99.04%. Time: 28.265289306640625s
[2024-07-10 11:56:32,357] [UTL] [INFO] 	Epoch 2 Accuracy 99.06% / Best Accuracy: 99.06%. Time: 27.669138431549072s
[2024-07-10 11:57:00,118] [UTL] [INFO] 	Epoch 3 Accuracy 99.05% / Best Accuracy: 99.06%. Time: 27.761221647262573s
[2024-07-10 11:57:27,860] [UTL] [INFO] 	Epoch 4 Accuracy 99.21% / Best Accuracy: 99.21%. Time: 27.742204666137695s
[2024-07-10 11:57:55,736] [UTL] [INFO] 	Epoch 5 Accuracy 99.20% / Best Accuracy: 99.21%. Time: 27.875325679779053s
[2024-07-10 11:58:23,615] [UTL] [INFO] 	Epoch 6 Accuracy 99.20% / Best Accuracy: 99.21%. Time: 27.877326488494873s
[2024-07-10 11:58:51,450] [UTL] [INFO] 	Epoch 7 Accuracy 99.25% / Best Accuracy: 99.25%. Time: 27.835289478302002s
[2024-07-10 11:59:19,241] [UTL] [INFO] 	Epoch 8 Accuracy 99.25% / Best Accuracy: 99.25%. Time: 27.790247917175293s
[2024-07-10 11:59:47,

### 256 => 128

In [2]:
# 128 Step

num_classes = ArrhythmiaLabels.size
model_name  = "resnet50"
model       = load_model(model_name)
model.fc    = nn.Linear(2048, num_classes)

# To resume training
# model = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/resnet18_ep10_i256_r0.3/resnet18_ep10_i256_r0.3.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [
    #TrainConfig(256, 0.3, 10),
    TrainConfig(128, 1.0, 30),
]
method = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")

    if config.window_size == 128:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5"
    else:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train_r{config.ratio}_{method}.h5"

    dataloader = build_dataloader(
        train_path    = train_path,
        test_path     = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5",
        transform     = None,
        batch_size    = 128,
        preload_train = True,
        preload_test  = (config.window_size == 128)
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Step-128/{save_name}"

    warm_up_dataloader(dataloader["train"])
    if config.window_size == 128:
        warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = (config.window_size == 128)
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

    del dataloader

[2024-07-10 20:55:30,662] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_train.h5
[2024-07-10 20:55:30,664] [DTSET] [INFO] 	Available Memory: 22.98 GiB
[2024-07-10 20:55:30,664] [DTSET] [INFO] 	Dataset Memory:   14.75 GiB


===== Window Size 128 =====


[2024-07-10 20:58:17,961] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 20:58:17,978] [DTSET] [INFO] 	Available Memory: 8.43 GiB
[2024-07-10 20:58:17,978] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 20:58:59,634] [UTL] [INFO] Begin finetuning...
[2024-07-10 20:59:31,318] [UTL] [INFO] 	Epoch 1 Accuracy 98.56% / Best Accuracy: 98.56%. Time: 31.682904958724976s
[2024-07-10 20:59:57,777] [UTL] [INFO] 	Epoch 2 Accuracy 98.72% / Best Accuracy: 98.72%. Time: 26.459038496017456s
[2024-07-10 21:00:24,248] [UTL] [INFO] 	Epoch 3 Accuracy 98.70% / Best Accuracy: 98.72%. Time: 26.470048904418945s
[2024-07-10 21:00:50,835] [UTL] [INFO] 	Epoch 4 Accuracy 98.80% / Best Accuracy: 98.80%. Time: 26.586154222488403s
[2024-07-10 21:01:17,439] [UTL] [INFO] 	Epoch 5 Accuracy 98.92% / Best Accuracy: 98.92%. Time: 26.60417103767395s
[2024-07-10 21:01:44,056] [UTL] [INFO] 	Epoch 6 Accuracy 98.90% / Best Accuracy: 98.92%.

In [3]:
# 64 Step

num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
model       = load_model(model_name)
model.fc    = nn.Linear(512, num_classes)

# To resume training
model = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/resnet18_ep10_i256_r0.3/resnet18_ep10_i256_r0.3.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [
    #TrainConfig(256, 0.3, 10),
    TrainConfig(192, 0.4, 10),
    TrainConfig(128, 1.0, 30),
]
method = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")

    if config.window_size == 128:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5"
    else:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train_r{config.ratio}_{method}.h5"

    dataloader = build_dataloader(
        train_path    = train_path,
        test_path     = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5",
        transform     = None,
        batch_size    = 128,
        preload_train = True,
        preload_test  = (config.window_size == 128)
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Step-64/{save_name}"

    warm_up_dataloader(dataloader["train"])
    if config.window_size == 128:
        warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = (config.window_size == 128)
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

    del dataloader

[2024-07-10 21:12:31,314] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-192/image_unfiltered_i192_train_r0.4_balance.h5
[2024-07-10 21:12:31,376] [DTSET] [INFO] 	Available Memory: 22.71 GiB
[2024-07-10 21:12:31,376] [DTSET] [INFO] 	Dataset Memory:   13.28 GiB


===== Window Size 192 =====


[2024-07-10 21:14:36,786] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 21:14:36,799] [DTSET] [INFO] 	Available Memory: 9.46 GiB
[2024-07-10 21:14:36,800] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 21:14:49,827] [UTL] [INFO] Begin finetuning...
[2024-07-10 21:15:09,451] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.624327898025513s
[2024-07-10 21:15:28,486] [UTL] [INFO] 	Epoch 2 Finished. Time: 19.033292531967163s
[2024-07-10 21:15:47,565] [UTL] [INFO] 	Epoch 3 Finished. Time: 19.079333782196045s
[2024-07-10 21:16:06,705] [UTL] [INFO] 	Epoch 4 Finished. Time: 19.13938879966736s
[2024-07-10 21:16:25,869] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.164411783218384s
[2024-07-10 21:16:45,046] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.177423000335693s
[2024-07-10 21:17:04,243] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.196441411972046s
[2024-07-10 21:17:23,446] [UTL] [INFO] 	Epoch 8 Finished. Time: 19.20344758

===== Window Size 128 =====


[2024-07-10 21:20:48,586] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 21:20:48,601] [DTSET] [INFO] 	Available Memory: 7.84 GiB
[2024-07-10 21:20:48,601] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 21:21:26,317] [UTL] [INFO] Begin finetuning...
[2024-07-10 21:21:52,553] [UTL] [INFO] 	Epoch 1 Accuracy 98.75% / Best Accuracy: 98.75%. Time: 26.235836267471313s
[2024-07-10 21:22:18,791] [UTL] [INFO] 	Epoch 2 Accuracy 98.80% / Best Accuracy: 98.80%. Time: 26.23783779144287s
[2024-07-10 21:22:45,054] [UTL] [INFO] 	Epoch 3 Accuracy 98.88% / Best Accuracy: 98.88%. Time: 26.262860536575317s
[2024-07-10 21:23:11,397] [UTL] [INFO] 	Epoch 4 Accuracy 98.95% / Best Accuracy: 98.95%. Time: 26.34193253517151s
[2024-07-10 21:23:37,827] [UTL] [INFO] 	Epoch 5 Accuracy 98.78% / Best Accuracy: 98.95%. Time: 26.430012941360474s
[2024-07-10 21:24:04,264] [UTL] [INFO] 	Epoch 6 Accuracy 98.96% / Best Accuracy: 98.96%. 

In [3]:
# 32 Step

num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
# model       = load_model(model_name)
# model.fc    = nn.Linear(512, num_classes)

# To resume training
model = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/Step-32/Epoch-20/resnet18_ep20_i256_r0.3/resnet18_ep20_i256_r0.3.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [
    # TrainConfig(256, 0.3, 20),
    TrainConfig(224, 0.3, 20),
    TrainConfig(192, 0.4, 20),
    TrainConfig(160, 0.5, 20),
    TrainConfig(128, 1.0, 30),
]
method = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")

    if config.window_size == 128:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5"
    else:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train_r{config.ratio}_{method}.h5"

    dataloader = build_dataloader(
        train_path    = train_path,
        test_path     = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5",
        transform     = None,
        batch_size    = 128,
        preload_train = True,
        preload_test  = (config.window_size == 128)
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Step-32/Epoch-20/{save_name}"

    warm_up_dataloader(dataloader["train"])
    if config.window_size == 128:
        warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = (config.window_size == 128) # Only evaluate on 128
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

    del dataloader

[2024-07-10 21:58:44,275] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-224/image_unfiltered_i224_train_r0.3_balance.h5
[2024-07-10 21:58:44,277] [DTSET] [INFO] 	Available Memory: 23.11 GiB
[2024-07-10 21:58:44,278] [DTSET] [INFO] 	Dataset Memory:   13.55 GiB


===== Window Size 224 =====


[2024-07-10 22:00:39,662] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 22:00:39,663] [DTSET] [INFO] 	Available Memory: 9.84 GiB
[2024-07-10 22:00:39,664] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 22:00:48,757] [UTL] [INFO] Begin finetuning...
[2024-07-10 22:01:08,566] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.808088302612305s
[2024-07-10 22:01:28,158] [UTL] [INFO] 	Epoch 2 Finished. Time: 19.591800212860107s
[2024-07-10 22:01:47,832] [UTL] [INFO] 	Epoch 3 Finished. Time: 19.67287278175354s
[2024-07-10 22:02:07,589] [UTL] [INFO] 	Epoch 4 Finished. Time: 19.75695013999939s
[2024-07-10 22:02:27,405] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.81600332260132s
[2024-07-10 22:02:47,236] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.83101725578308s
[2024-07-10 22:03:07,074] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.83802342414856s
[2024-07-10 22:03:26,931] [UTL] [INFO] 	Epoch 8 Finished. Time: 19.857040405273

===== Window Size 192 =====


[2024-07-10 22:09:32,445] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 22:09:32,459] [DTSET] [INFO] 	Available Memory: 10.10 GiB
[2024-07-10 22:09:32,459] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 22:09:42,942] [UTL] [INFO] Begin finetuning...
[2024-07-10 22:10:02,734] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.790789127349854s
[2024-07-10 22:10:22,084] [UTL] [INFO] 	Epoch 2 Finished. Time: 19.350579500198364s
[2024-07-10 22:10:41,491] [UTL] [INFO] 	Epoch 3 Finished. Time: 19.40663242340088s
[2024-07-10 22:11:00,956] [UTL] [INFO] 	Epoch 4 Finished. Time: 19.463682889938354s
[2024-07-10 22:11:20,469] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.512728452682495s
[2024-07-10 22:11:39,969] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.499715566635132s
[2024-07-10 22:11:59,520] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.549761533737183s
[2024-07-10 22:12:19,098] [UTL] [INFO] 	Epoch 8 Finished. Time: 19.5777876

===== Window Size 160 =====


[2024-07-10 22:18:11,090] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 22:18:11,091] [DTSET] [INFO] 	Available Memory: 11.80 GiB
[2024-07-10 22:18:11,092] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 22:18:22,819] [UTL] [INFO] Begin finetuning...
[2024-07-10 22:18:41,046] [UTL] [INFO] 	Epoch 1 Finished. Time: 18.226884365081787s
[2024-07-10 22:18:58,939] [UTL] [INFO] 	Epoch 2 Finished. Time: 17.893256425857544s
[2024-07-10 22:19:16,948] [UTL] [INFO] 	Epoch 3 Finished. Time: 18.008361339569092s
[2024-07-10 22:19:34,978] [UTL] [INFO] 	Epoch 4 Finished. Time: 18.02837872505188s
[2024-07-10 22:19:52,971] [UTL] [INFO] 	Epoch 5 Finished. Time: 17.99234700202942s
[2024-07-10 22:20:11,023] [UTL] [INFO] 	Epoch 6 Finished. Time: 18.052401065826416s
[2024-07-10 22:20:29,104] [UTL] [INFO] 	Epoch 7 Finished. Time: 18.07942533493042s
[2024-07-10 22:20:47,195] [UTL] [INFO] 	Epoch 8 Finished. Time: 18.090435981

===== Window Size 128 =====


[2024-07-10 22:27:11,976] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 22:27:11,991] [DTSET] [INFO] 	Available Memory: 9.33 GiB
[2024-07-10 22:27:11,992] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 22:27:54,027] [UTL] [INFO] Begin finetuning...
[2024-07-10 22:28:21,957] [UTL] [INFO] 	Epoch 1 Accuracy 98.87% / Best Accuracy: 98.87%. Time: 27.929577112197876s
[2024-07-10 22:28:49,036] [UTL] [INFO] 	Epoch 2 Accuracy 98.72% / Best Accuracy: 98.87%. Time: 27.077601194381714s
[2024-07-10 22:29:16,225] [UTL] [INFO] 	Epoch 3 Accuracy 98.92% / Best Accuracy: 98.92%. Time: 27.188700914382935s
[2024-07-10 22:29:43,485] [UTL] [INFO] 	Epoch 4 Accuracy 98.98% / Best Accuracy: 98.98%. Time: 27.259767055511475s
[2024-07-10 22:30:10,739] [UTL] [INFO] 	Epoch 5 Accuracy 99.11% / Best Accuracy: 99.11%. Time: 27.253760814666748s
[2024-07-10 22:30:38,138] [UTL] [INFO] 	Epoch 6 Accuracy 99.13% / Best Accuracy: 99.13%

In [5]:
# 16 Step

num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
# model       = load_model(model_name)
# model.fc    = nn.Linear(512, num_classes)

# To resume training
model = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/More/resnet18_ep10_i256_r0.3/resnet18_ep10_i256_r0.3.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [
    #TrainConfig(256, 0.3, 10),
    TrainConfig(240, 0.3, 10),
    TrainConfig(224, 0.3, 10),
    TrainConfig(208, 0.4, 10),
    TrainConfig(192, 0.4, 10),
    TrainConfig(176, 0.5, 10),
    TrainConfig(160, 0.5, 10),
    TrainConfig(144, 0.6, 10),
    TrainConfig(128, 1.0, 30),
]
method = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")

    if config.window_size == 128:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5"
    else:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train_r{config.ratio}_{method}.h5"

    dataloader = build_dataloader(
        train_path    = train_path,
        test_path     = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5",
        transform     = None,
        batch_size    = 128,
        preload_train = True,
        preload_test  = (config.window_size == 128)
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Step-16/{save_name}"

    warm_up_dataloader(dataloader["train"])
    if config.window_size == 128:
        warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = (config.window_size == 128)
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

    del dataloader

[2024-07-10 16:49:34,209] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-240/image_unfiltered_i240_train_r0.3_balance.h5
[2024-07-10 16:49:34,211] [DTSET] [INFO] 	Available Memory: 21.63 GiB
[2024-07-10 16:49:34,211] [DTSET] [INFO] 	Dataset Memory:   15.56 GiB


===== Window Size 240 =====


[2024-07-10 16:51:32,125] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 16:51:32,127] [DTSET] [INFO] 	Available Memory: 6.65 GiB
[2024-07-10 16:51:32,127] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 16:51:41,360] [UTL] [INFO] Begin finetuning...
[2024-07-10 16:52:04,886] [UTL] [INFO] 	Epoch 1 Finished. Time: 23.524114847183228s
[2024-07-10 16:52:28,083] [UTL] [INFO] 	Epoch 2 Finished. Time: 23.196073055267334s
[2024-07-10 16:52:51,369] [UTL] [INFO] 	Epoch 3 Finished. Time: 23.28615713119507s
[2024-07-10 16:53:14,757] [UTL] [INFO] 	Epoch 4 Finished. Time: 23.387248039245605s
[2024-07-10 16:53:38,166] [UTL] [INFO] 	Epoch 5 Finished. Time: 23.40926718711853s
[2024-07-10 16:54:01,729] [UTL] [INFO] 	Epoch 6 Finished. Time: 23.561720609664917s
[2024-07-10 16:54:25,131] [UTL] [INFO] 	Epoch 7 Finished. Time: 23.402260780334473s
[2024-07-10 16:54:48,558] [UTL] [INFO] 	Epoch 8 Finished. Time: 23.425282716

===== Window Size 224 =====


[2024-07-10 16:57:32,299] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 16:57:32,321] [DTSET] [INFO] 	Available Memory: 8.65 GiB
[2024-07-10 16:57:32,322] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 16:57:44,108] [UTL] [INFO] Begin finetuning...
[2024-07-10 16:58:03,935] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.827564477920532s
[2024-07-10 16:58:23,410] [UTL] [INFO] 	Epoch 2 Finished. Time: 19.474918365478516s
[2024-07-10 16:58:42,514] [UTL] [INFO] 	Epoch 3 Finished. Time: 19.103755235671997s
[2024-07-10 16:59:02,216] [UTL] [INFO] 	Epoch 4 Finished. Time: 19.70089817047119s
[2024-07-10 16:59:21,860] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.64284610748291s
[2024-07-10 16:59:41,540] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.679879665374756s
[2024-07-10 17:00:01,216] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.67687749862671s
[2024-07-10 17:00:20,934] [UTL] [INFO] 	Epoch 8 Finished. Time: 19.7179141044

...

[2024-07-10 17:20:37,601] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 17:20:37,617] [DTSET] [INFO] 	Available Memory: 10.65 GiB
[2024-07-10 17:20:37,618] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 17:20:49,616] [UTL] [INFO] Begin finetuning...
[2024-07-10 17:21:07,899] [UTL] [INFO] 	Epoch 1 Finished. Time: 18.282028913497925s
[2024-07-10 17:21:25,653] [UTL] [INFO] 	Epoch 2 Finished. Time: 17.75212788581848s
[2024-07-10 17:21:43,505] [UTL] [INFO] 	Epoch 3 Finished. Time: 17.851218461990356s
[2024-07-10 17:22:01,376] [UTL] [INFO] 	Epoch 4 Finished. Time: 17.871236085891724s
[2024-07-10 17:22:19,314] [UTL] [INFO] 	Epoch 5 Finished. Time: 17.938297510147095s
[2024-07-10 17:22:37,242] [UTL] [INFO] 	Epoch 6 Finished. Time: 17.927287578582764s
[2024-07-10 17:22:55,246] [UTL] [INFO] 	Epoch 7 Finished. Time: 18.00335669517517s
[2024-07-10 17:23:13,201] [UTL] [INFO] 	Epoch 8 Finished. Time: 17.95531272

===== Window Size 144 =====


[2024-07-10 17:25:32,178] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 17:25:32,197] [DTSET] [INFO] 	Available Memory: 11.15 GiB
[2024-07-10 17:25:32,198] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 17:25:45,133] [UTL] [INFO] Begin finetuning...
[2024-07-10 17:26:04,422] [UTL] [INFO] 	Epoch 1 Finished. Time: 19.2875235080719s
[2024-07-10 17:26:23,223] [UTL] [INFO] 	Epoch 2 Finished. Time: 18.800080060958862s
[2024-07-10 17:26:41,956] [UTL] [INFO] 	Epoch 3 Finished. Time: 18.733019828796387s
[2024-07-10 17:27:00,897] [UTL] [INFO] 	Epoch 4 Finished. Time: 18.940208196640015s
[2024-07-10 17:27:20,004] [UTL] [INFO] 	Epoch 5 Finished. Time: 19.107359170913696s
[2024-07-10 17:27:39,226] [UTL] [INFO] 	Epoch 6 Finished. Time: 19.220481872558594s
[2024-07-10 17:27:58,351] [UTL] [INFO] 	Epoch 7 Finished. Time: 19.124375104904175s
[2024-07-10 17:28:17,281] [UTL] [INFO] 	Epoch 8 Finished. Time: 18.92919754

===== Window Size 128 =====


[2024-07-10 17:31:43,675] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-10 17:31:43,691] [DTSET] [INFO] 	Available Memory: 8.15 GiB
[2024-07-10 17:31:43,692] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-10 17:32:38,249] [UTL] [INFO] Begin finetuning...
[2024-07-10 17:33:15,573] [UTL] [INFO] 	Epoch 1 Accuracy 98.97% / Best Accuracy: 98.97%. Time: 37.324442625045776s
[2024-07-10 17:33:48,666] [UTL] [INFO] 	Epoch 2 Accuracy 99.02% / Best Accuracy: 99.02%. Time: 33.0930655002594s
[2024-07-10 17:34:21,547] [UTL] [INFO] 	Epoch 3 Accuracy 99.16% / Best Accuracy: 99.16%. Time: 32.88087320327759s
[2024-07-10 17:34:54,456] [UTL] [INFO] 	Epoch 4 Accuracy 99.11% / Best Accuracy: 99.16%. Time: 32.90889930725098s
[2024-07-10 17:35:27,396] [UTL] [INFO] 	Epoch 5 Accuracy 99.18% / Best Accuracy: 99.18%. Time: 32.93992638587952s
[2024-07-10 17:36:00,266] [UTL] [INFO] 	Epoch 6 Accuracy 99.22% / Best Accuracy: 99.22%. Tim

In [2]:
# 8 Step

num_classes = ArrhythmiaLabels.size
model_name  = "resnet50"
# model       = load_model(model_name)
# model.fc    = nn.Linear(2048, num_classes)

# To resume training
model = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/ResNet50/resnet50_ep10_i200_r0.4/resnet50_ep10_i200_r0.4.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs", "batch_size"])
train_config = [
    # TrainConfig(256, 0.3, 10, 64),
    # TrainConfig(248, 0.3, 10, 64),
    # TrainConfig(240, 0.3, 10, 64),
    # TrainConfig(232, 0.3, 10, 64),
    # TrainConfig(224, 0.3, 10, 64),
    # TrainConfig(216, 0.4, 10, 64),
    # TrainConfig(208, 0.4, 10, 64),
    # TrainConfig(200, 0.4, 10, 64),
    TrainConfig(192, 0.4, 10, 64),
    TrainConfig(184, 0.4, 10, 64),
    TrainConfig(176, 0.5, 10, 128),
    TrainConfig(168, 0.5, 10, 128),
    TrainConfig(160, 0.5, 10, 128),
    TrainConfig(152, 0.6, 10, 128),
    TrainConfig(144, 0.6, 10, 128),
    TrainConfig(136, 0.6, 10, 128),
    TrainConfig(128, 1.0, 30, 128),
]
method = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")

    if config.window_size == 128:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5"
    else:
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train_r{config.ratio}_{method}.h5"

    dataloader = build_dataloader(
        train_path    = train_path,
        test_path     = f"../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5",
        transform     = None,
        batch_size    = config.batch_size,
        preload_train = True,
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/ResNet50/{save_name}"

    warm_up_dataloader(dataloader["train"])
    if config.window_size == 128:
        warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = (config.window_size == 128)
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

    del dataloader

[2024-07-16 14:46:24,662] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-192/image_unfiltered_i192_train_r0.4_balance.h5
[2024-07-16 14:46:24,664] [DTSET] [INFO] 	Available Memory: 25.33 GiB
[2024-07-16 14:46:24,665] [DTSET] [INFO] 	Dataset Memory:   13.28 GiB
[2024-07-16 14:46:24,665] [DTSET] [INFO] 	Preloading images and labels


===== Window Size 192 =====


[2024-07-16 14:47:02,197] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-16 14:47:02,211] [DTSET] [INFO] 	Available Memory: 12.02 GiB
[2024-07-16 14:47:02,211] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-16 14:47:08,665] [CLASS] [INFO] Begin finetuning...
[2024-07-16 14:48:07,818] [CLASS] [INFO] 	Epoch 1 Finished. Time: 59.152076721191406s
[2024-07-16 14:49:03,906] [CLASS] [INFO] 	Epoch 2 Finished. Time: 56.08792161941528s
[2024-07-16 14:50:00,158] [CLASS] [INFO] 	Epoch 3 Finished. Time: 56.25107002258301s
[2024-07-16 14:50:56,516] [CLASS] [INFO] 	Epoch 4 Finished. Time: 56.35816740989685s
[2024-07-16 14:51:52,944] [CLASS] [INFO] 	Epoch 5 Finished. Time: 56.42823052406311s
[2024-07-16 14:52:49,402] [CLASS] [INFO] 	Epoch 6 Finished. Time: 56.458258867263794s
[2024-07-16 14:53:45,923] [CLASS] [INFO] 	Epoch 7 Finished. Time: 56.52031397819519s
[2024-07-16 14:54:42,397] [CLASS] [INFO] 	Epoch 8 Finished. Ti

===== Window Size 184 =====


[2024-07-16 14:58:20,457] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-16 14:58:20,459] [DTSET] [INFO] 	Available Memory: 12.65 GiB
[2024-07-16 14:58:20,459] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-16 14:58:27,826] [CLASS] [INFO] Begin finetuning...
[2024-07-16 14:59:23,759] [CLASS] [INFO] 	Epoch 1 Finished. Time: 55.93229937553406s
[2024-07-16 15:00:19,739] [CLASS] [INFO] 	Epoch 2 Finished. Time: 55.978822469711304s
[2024-07-16 15:01:15,841] [CLASS] [INFO] 	Epoch 3 Finished. Time: 56.10193467140198s
[2024-07-16 15:02:12,029] [CLASS] [INFO] 	Epoch 4 Finished. Time: 56.18701195716858s
[2024-07-16 15:03:08,321] [CLASS] [INFO] 	Epoch 5 Finished. Time: 56.292107582092285s
[2024-07-16 15:04:04,625] [CLASS] [INFO] 	Epoch 6 Finished. Time: 56.30411887168884s
[2024-07-16 15:05:00,938] [CLASS] [INFO] 	Epoch 7 Finished. Time: 56.312124729156494s
[2024-07-16 15:05:57,285] [CLASS] [INFO] 	Epoch 8 Finished. T

...

[2024-07-16 15:54:33,290] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-16 15:54:33,292] [DTSET] [INFO] 	Available Memory: 14.31 GiB
[2024-07-16 15:54:33,292] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-16 15:54:42,918] [CLASS] [INFO] Begin finetuning...
[2024-07-16 15:55:33,193] [CLASS] [INFO] 	Epoch 1 Finished. Time: 50.27527451515198s
[2024-07-16 15:56:23,461] [CLASS] [INFO] 	Epoch 2 Finished. Time: 50.26663613319397s
[2024-07-16 15:57:13,925] [CLASS] [INFO] 	Epoch 3 Finished. Time: 50.463815689086914s
[2024-07-16 15:58:04,478] [CLASS] [INFO] 	Epoch 4 Finished. Time: 50.55289673805237s
[2024-07-16 15:58:55,035] [CLASS] [INFO] 	Epoch 5 Finished. Time: 50.55690002441406s
[2024-07-16 15:59:45,604] [CLASS] [INFO] 	Epoch 6 Finished. Time: 50.5689115524292s
[2024-07-16 16:00:36,176] [CLASS] [INFO] 	Epoch 7 Finished. Time: 50.571913957595825s
[2024-07-16 16:01:26,767] [CLASS] [INFO] 	Epoch 8 Finished. Tim

===== Window Size 136 =====


[2024-07-16 16:04:35,920] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-16 16:04:35,922] [DTSET] [INFO] 	Available Memory: 15.40 GiB
[2024-07-16 16:04:35,923] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-16 16:04:45,589] [CLASS] [INFO] Begin finetuning...
[2024-07-16 16:05:33,203] [CLASS] [INFO] 	Epoch 1 Finished. Time: 47.61423468589783s
[2024-07-16 16:06:20,793] [CLASS] [INFO] 	Epoch 2 Finished. Time: 47.59020638465881s
[2024-07-16 16:07:08,594] [CLASS] [INFO] 	Epoch 3 Finished. Time: 47.799396991729736s
[2024-07-16 16:07:56,467] [CLASS] [INFO] 	Epoch 4 Finished. Time: 47.87246322631836s
[2024-07-16 16:08:44,319] [CLASS] [INFO] 	Epoch 5 Finished. Time: 47.851444244384766s
[2024-07-16 16:09:32,198] [CLASS] [INFO] 	Epoch 6 Finished. Time: 47.87946939468384s
[2024-07-16 16:10:20,089] [CLASS] [INFO] 	Epoch 7 Finished. Time: 47.89057445526123s
[2024-07-16 16:11:08,013] [CLASS] [INFO] 	Epoch 8 Finished. Ti

===== Window Size 128 =====


[2024-07-16 16:15:33,172] [DTSET] [INFO] Building ECG Dataset from ../Data/MIT-BIH-Raw/Datasets/Resolution-128/image_unfiltered_i128_test.h5
[2024-07-16 16:15:33,188] [DTSET] [INFO] 	Available Memory: 10.66 GiB
[2024-07-16 16:15:33,188] [DTSET] [INFO] 	Dataset Memory:   3.69 GiB
[2024-07-16 16:16:18,954] [CLASS] [INFO] Begin finetuning...
[2024-07-16 16:17:30,588] [CLASS] [INFO] 	Epoch 1 Accuracy 99.13% / Best Accuracy: 99.13%. Time: 71.63420987129211s
[2024-07-16 16:18:41,621] [CLASS] [INFO] 	Epoch 2 Accuracy 99.17% / Best Accuracy: 99.17%. Time: 71.03248977661133s
[2024-07-16 16:19:52,740] [CLASS] [INFO] 	Epoch 3 Accuracy 99.28% / Best Accuracy: 99.28%. Time: 71.11956930160522s
[2024-07-16 16:21:03,852] [CLASS] [INFO] 	Epoch 4 Accuracy 99.30% / Best Accuracy: 99.30%. Time: 71.1115608215332s
[2024-07-16 16:22:14,914] [CLASS] [INFO] 	Epoch 5 Accuracy 99.30% / Best Accuracy: 99.30%. Time: 71.06151795387268s
[2024-07-16 16:23:26,056] [CLASS] [INFO] 	Epoch 6 Accuracy 99.35% / Best Accurac

### 128 => 96

In [4]:
num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
model       = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/Large/resnet18_ep30_i128_r1.0/resnet18_ep30_i128_r1.0.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [TrainConfig(96, 1.0, 30)]
method        = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")
    dataloader = build_dataloader(
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5",
        test_path  = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_test.h5",
        transform  = None,
        batch_size = 128
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Large/{save_name}"

    warm_up_dataloader(dataloader["train"])
    warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = True
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

===== Window Size 96 =====


[2024-07-06 16:01:42,492] [UTL] [INFO] Begin finetuning...
[2024-07-06 16:02:02,771] [UTL] [INFO] 	Epoch 1 Accuracy 98.50% / Best Accuracy: 98.50%. Time: 20.278424501419067s
[2024-07-06 16:02:22,864] [UTL] [INFO] 	Epoch 2 Accuracy 98.60% / Best Accuracy: 98.60%. Time: 20.093254566192627s
[2024-07-06 16:02:42,875] [UTL] [INFO] 	Epoch 3 Accuracy 98.59% / Best Accuracy: 98.60%. Time: 20.011181354522705s
[2024-07-06 16:03:02,921] [UTL] [INFO] 	Epoch 4 Accuracy 98.71% / Best Accuracy: 98.71%. Time: 20.046212911605835s
[2024-07-06 16:03:23,010] [UTL] [INFO] 	Epoch 5 Accuracy 98.80% / Best Accuracy: 98.80%. Time: 20.088250875473022s
[2024-07-06 16:03:42,997] [UTL] [INFO] 	Epoch 6 Accuracy 98.68% / Best Accuracy: 98.80%. Time: 19.98615837097168s
[2024-07-06 16:04:02,884] [UTL] [INFO] 	Epoch 7 Accuracy 98.68% / Best Accuracy: 98.80%. Time: 19.886067152023315s
[2024-07-06 16:04:22,774] [UTL] [INFO] 	Epoch 8 Accuracy 98.66% / Best Accuracy: 98.80%. Time: 19.890070915222168s
[2024-07-06 16:04:42,7

### 96 => 64

In [5]:
num_classes = ArrhythmiaLabels.size
model_name  = "resnet18"
model       = load_model_from_pretrained(model_name, f"../Pretrained/ECG-Raw/Window-Shrinking/balance/Large/resnet18_ep30_i96_r1.0/resnet18_ep30_i96_r1.0.pth", num_classes)

TrainConfig  = namedtuple("TrainConfig", ["window_size", "ratio", "epochs"])
train_config = [TrainConfig(64, 1.0, 30)]
method       = "balance"

for config in train_config:
    print(f"===== Window Size {config.window_size} =====")
    dataloader = build_dataloader(
        train_path = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_train.h5",
        test_path  = f"../Data/MIT-BIH-Raw/Datasets/Resolution-{config.window_size}/image_unfiltered_i{config.window_size}_test.h5",
        transform  = None,
        batch_size = 128
    )

    save_name = f"{model_name}_ep{config.epochs}_i{config.window_size}_r{config.ratio}"
    save_dir  = f"../Pretrained/ECG-Raw/Window-Shrinking/{method}/Large/{save_name}"

    warm_up_dataloader(dataloader["train"])
    warm_up_dataloader(dataloader["test"])

    training_stats = finetune(
        model      = model,
        epochs     = config.epochs,
        dataloader = dataloader,
        save_path  = f"{save_dir}/{save_name}.pth",
        do_eval    = True
    )

    training_stats.serialize(f"{save_dir}/stats.txt")

===== Window Size 64 =====


[2024-07-06 16:33:14,374] [UTL] [INFO] Begin finetuning...
[2024-07-06 16:33:30,249] [UTL] [INFO] 	Epoch 1 Accuracy 97.76% / Best Accuracy: 97.76%. Time: 15.873305320739746s
[2024-07-06 16:33:45,953] [UTL] [INFO] 	Epoch 2 Accuracy 97.91% / Best Accuracy: 97.91%. Time: 15.704267740249634s
[2024-07-06 16:34:01,681] [UTL] [INFO] 	Epoch 3 Accuracy 98.12% / Best Accuracy: 98.12%. Time: 15.728290796279907s
[2024-07-06 16:34:17,374] [UTL] [INFO] 	Epoch 4 Accuracy 97.94% / Best Accuracy: 98.12%. Time: 15.69225549697876s
[2024-07-06 16:34:33,053] [UTL] [INFO] 	Epoch 5 Accuracy 98.09% / Best Accuracy: 98.12%. Time: 15.679245471954346s
[2024-07-06 16:34:48,754] [UTL] [INFO] 	Epoch 6 Accuracy 98.01% / Best Accuracy: 98.12%. Time: 15.700262308120728s
[2024-07-06 16:35:04,477] [UTL] [INFO] 	Epoch 7 Accuracy 98.09% / Best Accuracy: 98.12%. Time: 15.723286628723145s
[2024-07-06 16:35:20,248] [UTL] [INFO] 	Epoch 8 Accuracy 98.12% / Best Accuracy: 98.12%. Time: 15.770328283309937s
[2024-07-06 16:35:36,1